In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.vision import *
from fastai.callbacks.hooks import *
from fastai.utils.mem import *

In [3]:
camvid = untar_data(URLs.CAMVID)
path_lbl = camvid/'labels'
path_img = camvid/'images'
path_vidFrames = camvid/'commaImages'

# All commaAI images that are segmented are going to be in here 
path_Predicted = camvid/'predictedImages'

In [4]:
camvid.ls()

[PosixPath('/home/jupyter/.fastai/data/camvid/predictedImages'),
 PosixPath('/home/jupyter/.fastai/data/camvid/export.pkl'),
 PosixPath('/home/jupyter/.fastai/data/camvid/images'),
 PosixPath('/home/jupyter/.fastai/data/camvid/codes.txt'),
 PosixPath('/home/jupyter/.fastai/data/camvid/labels'),
 PosixPath('/home/jupyter/.fastai/data/camvid/base92.pkl'),
 PosixPath('/home/jupyter/.fastai/data/camvid/valid.txt'),
 PosixPath('/home/jupyter/.fastai/data/camvid/commaImages')]

# Video (train.mp4) to frames.jpg

- Use this only if you want to create new images
- Remember to check WHERE your images are being saved

In [6]:
# import cv2
# vidcap = cv2.VideoCapture('/home/jupyter/.fastai/data/camvid/commaImages/train.mp4')

#/home/jupyter/.fastai/data/camvid/commaImages

# success,image = vidcap.read()

# count = 0

# while success:
#     # save frame as jpg file
#     cv2.imwrite("frame%d.jpg" % count, image)

#     success, image = vidcap.read()
#     
#     if count % 1000 == 0 or count == 20400:
#         print("read a new frame: ", success)
#     count += 1

# /home/jupyter/.fastai/data/commaImages

# Data Prep, continued

In [5]:
codes = np.loadtxt(camvid/'codes.txt', dtype=str);  codes

array(['Animal', 'Archway', 'Bicyclist', 'Bridge', 'Building', 'Car', 'CartLuggagePram', 'Child', 'Column_Pole',
       'Fence', 'LaneMkgsDriv', 'LaneMkgsNonDriv', 'Misc_Text', 'MotorcycleScooter', 'OtherMoving', 'ParkingBlock',
       'Pedestrian', 'Road', 'RoadShoulder', 'Sidewalk', 'SignSymbol', 'Sky', 'SUVPickupTruck', 'TrafficCone',
       'TrafficLight', 'Train', 'Tree', 'Truck_Bus', 'Tunnel', 'VegetationMisc', 'Void', 'Wall'], dtype='<U17')

In [6]:
import skimage
import time

get_y_fn = lambda x: path_lbl/f'{x.stem}_P{x.suffix}'

name2id = {v:k for k,v in enumerate(codes)}; name2id

{'Animal': 0,
 'Archway': 1,
 'Bicyclist': 2,
 'Bridge': 3,
 'Building': 4,
 'Car': 5,
 'CartLuggagePram': 6,
 'Child': 7,
 'Column_Pole': 8,
 'Fence': 9,
 'LaneMkgsDriv': 10,
 'LaneMkgsNonDriv': 11,
 'Misc_Text': 12,
 'MotorcycleScooter': 13,
 'OtherMoving': 14,
 'ParkingBlock': 15,
 'Pedestrian': 16,
 'Road': 17,
 'RoadShoulder': 18,
 'Sidewalk': 19,
 'SignSymbol': 20,
 'Sky': 21,
 'SUVPickupTruck': 22,
 'TrafficCone': 23,
 'TrafficLight': 24,
 'Train': 25,
 'Tree': 26,
 'Truck_Bus': 27,
 'Tunnel': 28,
 'VegetationMisc': 29,
 'Void': 30,
 'Wall': 31}

In [7]:
void_code = name2id['Void']

def acc_camvid(input, target):
    target = target.squeeze(1)
    mask = target != void_code
    return (input.argmax(dim=1)[mask] == target[mask]).float().mean()

In [8]:
# Get images file names created from CommaAi video
frames = get_image_files(path_vidFrames)

# Load learner, predict images from frames

In [9]:
# Loading a previously trained model for segmentation
learn= load_learner(camvid)

In [10]:
# Get image files from 
frames = get_image_files(path_vidFrames)

In [11]:
len(frames)

20400

In [18]:
# img = open_image(path_vidFrames/'frame1015.jpg')

In [ ]:
count = 200
arr = []
while count < 205:
    img = open_image(path_vidFrames/f'frame{count}.jpg')
    
    # Crop the TOP and BOTTOM of images (top is just sky, bottom is dashboard)
    crop_pad(img, (390,640), 'border', 0.,0.).show()

    # Alter the BRIGHTNESS
    brightness(img, 0.71)

    pred = learn.predict(img)
    pred[0].show(figsize=(10,10))
    
    # *** Figure out how to store the segmented images...Array, .jpg?
    # Save the image in a FOLDER or ARRAY
    arr.append(pred)
    
    # Increase count, move on to next image
    count += 1

### Use this to view your SEGMENTED images

In [ ]:
# for i in arr:
#     print(i[0].show(figsize=(10,10)))